- In this notebook we tried to extend the base dataset using Tourpedia dataset form Paris attractions.
- We first a cleaned version of Tourpedia dataset from data.mendeley.com to indentify the subCategory related to our base dataset using human inspection.
- Then we filtered the main Tourpedia dataset by these related subCategories to get a smaller dataset that is more relevant to our base dataset.
- Finally, we use the the details API and reviews API to extend the filtered Tourpedia dataset with more information and create a review dataset.

In [1]:
import pandas as pd

In [2]:
path = "Tourpedia/Paris.csv"

df_tp_mendeley = pd.read_csv(
    path,
    sep=";",
)

df_tp_mendeley.subCategory.unique().tolist()


# df_tp_mendeley_base_related = df_tp_mendeley[df_tp_mendeley["subCategory"].isin(kept_strict)].reset_index(drop=True)
# print(df_tp_mendeley_base_related.shape)
# df_tp_mendeley_base_related.head(2)

['Hotel Bar',
 'Hotel',
 'Wine Bar',
 'Music Venue',
 'Pub',
 'Bar',
 'Cocktail Bar',
 'Sports Bar',
 'Nightclub',
 'Rock Club',
 'Lounge',
 'Church',
 'Lake',
 'Cemetery',
 'Plaza',
 'Park',
 'Event Space',
 'Temple',
 'Mosque',
 'Historic Site',
 'Garden',
 'Bridge',
 'Museum',
 'Zoo',
 'Art Museum',
 'Harbor / Marina',
 'Island',
 'Art Gallery',
 'History Museum',
 'Theater',
 'Comedy Club',
 'Concert Hall',
 'Strip Club',
 'Multiplex',
 'Arts & Entertainment',
 'Pool',
 'Movie Theater',
 'Stadium',
 'Gay Bar',
 'Radio Station',
 'Neighborhood',
 'Performing Arts Venue',
 'Indie Movie Theater',
 'Public Art',
 'Farm',
 'Scenic Lookout',
 'River',
 'Opera House',
 'Dance Studio',
 'Auditorium',
 'Athletics & Sports',
 'General Entertainment',
 'Hostel',
 'Racetrack',
 'Golf Course',
 'Other Great Outdoors',
 'Brewery',
 'Track Stadium',
 'Other Nightlife',
 'Pizza Place',
 'Trail',
 'Science Museum',
 'Tennis Court',
 'Tennis',
 'Sculpture Garden',
 'Bowling Alley',
 'Military Base',

In [3]:
related_subCategory_set = {
    "Church",
    "Temple",
    "Mosque",
    "Historic Site",
    "Museum",
    "Art Museum",
    "History Museum",
    "Science Museum",
    "Art Gallery",
    "Public Art",
    "Monument / Landmark",
    "Cemetery",
    "Garden",
    "Sculpture Garden",
    "Park",
    "Plaza",
    "Bridge",
}

In [4]:
df_tp = pd.read_csv("Tourpedia/paris-attraction.csv")
df_tp.id
print(df_tp.shape)
df_tp.head(2)

(4351, 12)


,address,category,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews
0,8 rue de Navarin,attraction,83256,48.87961034033,2.3393825451216,Paris,Hôtel Amour,4ac79395f964a52021b820e3,0,Hotel Bar,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...
1,49 Rue Pierre Charron,attraction,83321,48.869145508329,2.3021912516801,Paris,Pershing Hall,4adcda03f964a520c03121e3,0,Hotel,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...


In [5]:
df_tp_base_related =  df_tp[df_tp.subCategory.isin(related_subCategory_set)].copy().reset_index(drop=True)
print(df_tp_base_related.shape)
df_tp_base_related.head(2)

(486, 12)


,address,category,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews
0,Place de la Madeleine,attraction,83456,48.869753812208,2.3242735862732,Paris,Église de la Madeleine,4adcda09f964a520013421e3,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...
1,20 Avenue Rachel,attraction,83461,48.887548749415,2.3304319381714,Paris,Cimetière de Montmartre,4adcda09f964a5200a3421e3,0,Cemetery,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...


In [6]:
import requests
import time
import pathlib

df_tp_base_related_reviews_addr = pathlib.Path("Tourpedia/paris-attraction-base-related-reviews.csv")
df_tp_base_related_with_details_addr = pathlib.Path("Tourpedia/paris-attraction-base-related.csv")

if pathlib.Path.exists(df_tp_base_related_reviews_addr) and pathlib.Path.exists(df_tp_base_related_with_details_addr):
    df_tp_base_related_reviews = pd.read_csv(df_tp_base_related_reviews_addr)
    df_tp_base_related_reviews.time = pd.to_datetime(df_tp_base_related_reviews.time)
    df_tp_base_related_with_details = pd.read_csv(df_tp_base_related_with_details_addr)
else:
    df_tp_base_related_reviews = pd.DataFrame(columns=[
        'monument_id',
        'language',
        'polarity',
        'rating',
        'source',
        'text',
        'time',
        'wordsCount',
        'details_url'
    ])
    df_tp_base_related_with_details = df_tp_base_related.copy()

    for index, row in df_tp_base_related.iterrows():
        details_url = row['details']
        reviews_url = row['reviews']
        
        res_details = requests.get(details_url)
        while res_details.status_code != 200:
            time.sleep(1)
            res_details = requests.get(details_url)
        res_reveiews = requests.get(reviews_url)
        while res_reveiews.status_code != 200:
            time.sleep(1)
            res_reveiews = requests.get(reviews_url)
            
        df_tp_base_related_reviews = pd.concat([df_tp_base_related_reviews, pd.DataFrame([{
            'monument_id': row['id'],
            'language': review['language'],
            'polarity': review['polarity'],
            'rating': review['rating'],
            'source': review['source'],
            'text': review['text'],
            'time': review['time'],
            'wordsCount': review['wordsCount'],
            'details_url': review['details']
        } for review in res_reveiews.json()])], ignore_index=True)
        
        res_details_json = res_details.json()
        # by checking the following condition we confirm that all data are only comming form Foursquare
        # if res_details_json['external_urls']['Facebook'] != "" or res_details_json['external_urls']['GooglePlaces'] != "" or res_details_json['external_urls']['Booking'] != "":
        #     print(res_details_json)
        # break
        df_tp_base_related_with_details.at[index, 'description.en'] = res_details_json['description']['en']
        df_tp_base_related_with_details.at[index, 'description.fr'] = res_details_json['description']['fr']
        df_tp_base_related_with_details.at[index, 'Foursquare.url'] = res_details_json['external_urls']['Foursquare']
        df_tp_base_related_with_details.at[index, 'international_phone_number'] = res_details_json['international_phone_number']
        df_tp_base_related_with_details.at[index, 'phone_number'] = res_details_json['phone_number']
        df_tp_base_related_with_details.at[index, 'polarity'] = res_details_json['polarity']
        df_tp_base_related_with_details.at[index, 'services'] = str(res_details_json['services'])
        df_tp_base_related_with_details.at[index, 'Foursquare.usersCount'] = res_details_json['statistics']['Foursquare']['usersCount']
        df_tp_base_related_with_details.at[index, 'Foursquare.checkinsCount'] = res_details_json['statistics']['Foursquare']['checkinsCount']
        df_tp_base_related_with_details.at[index, 'Foursquare.tipCount'] = res_details_json['statistics']['Foursquare']['tipCount']
        df_tp_base_related_with_details.at[index, 'Foursquare.price'] = res_details_json['statistics']['Foursquare']['price']
        df_tp_base_related_with_details.at[index, 'Foursquare.likes'] = res_details_json['statistics']['Foursquare']['likes']
        df_tp_base_related_with_details.at[index, 'website'] = res_details_json['website']
        # based on our evaluation if the numReviews field is missing it has no reviews and if it is not has some reviews so we can set it to 0
        df_tp_base_related_with_details.at[index, 'numReviews'] = res_details_json.get('numReviews', 0) 
        
    # removing empty clomuns and converting dtypes 
    # we previously checked that all values are convertible to the target dtypes
    # and all of the dorpped columns are also either empty or have single unique value which is not useful for our purpose
    df_tp_base_related_with_details.drop(columns=[
        'international_phone_number', 
        'phone_number', 
        'services', 
        'description.fr', 
        'description.en',
        'category',
        'Foursquare.price'
    ], inplace=True)
    df_tp_base_related_with_details = df_tp_base_related_with_details.astype({
        'id': 'Int64',
        'lat': 'Float64',
        'lng': 'Float64',
        'polarity': 'Int64',
        'Foursquare.usersCount': 'Int64',
        'Foursquare.checkinsCount': 'Int64',
        'Foursquare.tipCount': 'Int64',
        'Foursquare.likes': 'Int64',
        'numReviews': 'Int64'
    })
    df_tp_base_related_reviews.drop(columns=[
        'rating',
        'source',
    ], inplace=True)
    df_tp_base_related_reviews = df_tp_base_related_reviews.astype({
        'monument_id': 'Int64',
        'polarity': 'Int64',
        'time': 'datetime64[ns]',
        'wordsCount': 'Int64'
    })
    # saving dataframes
    df_tp_base_related_reviews.to_csv(df_tp_base_related_reviews_addr, index=False)
    df_tp_base_related_with_details.to_csv(df_tp_base_related_with_details_addr, index=False)

# Extracted Fields from the Detials of df_tp
- description.en -> description
- external_urls.Foursquare
- international_phone_number
- phone_number
- polarity
- services (is a list)
- statistics.Foursquare.usersCount
- statistics.Foursquare.checkinsCount
- statistics.Foursquare.tipCount
- statistics.Foursquare.price
- statistics.Foursquare.likes
- website
- numReviews

In [7]:
# number of reviews consistency check
(
    df_tp_base_related_reviews.groupby('monument_id').size().loc[
        df_tp_base_related_with_details[df_tp_base_related_with_details['numReviews'] > 0]['id']
    ] == df_tp_base_related_with_details[df_tp_base_related_with_details['numReviews'] > 0].set_index('id')['numReviews']
).all()

np.True_

In [8]:
# polarity consistency check
import numpy as np

(
    df_tp_base_related_reviews.groupby('monument_id')['polarity'].apply(lambda x: np.floor(x.mean() + 0.5)).astype('Int64').loc[
        df_tp_base_related_with_details[df_tp_base_related_with_details['numReviews'] > 0]['id']
    ] == df_tp_base_related_with_details[df_tp_base_related_with_details['numReviews'] > 0].set_index('id')['polarity']
).all()

np.True_

In [9]:
print(df_tp_base_related_with_details.shape)
df_tp_base_related_with_details.head(2)

(486, 18)


,address,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews,Foursquare.url,Foursquare.usersCount,Foursquare.checkinsCount,Foursquare.tipCount,Foursquare.likes,website,numReviews
0,Place de la Madeleine,83456,48.869754,2.324274,Paris,Église de la Madeleine,4adcda09f964a520013421e3,4,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/%C3%A9glise-de-la-mad...,1956,2839,9,28,http://www.eglise-lamadeleine.com,9
1,20 Avenue Rachel,83461,48.887549,2.330432,Paris,Cimetière de Montmartre,4adcda09f964a5200a3421e3,6,Cemetery,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/cimeti%C3%A8re-de-mon...,636,1164,6,8,NaN,6


In [10]:
print(df_tp_base_related_reviews.shape)
df_tp_base_related_reviews.head(2)

(2613, 7)


,monument_id,language,polarity,text,time,wordsCount,details_url
0,83456,fr,5,Une des trois plus laides églises de Paris ! M...,2011-08-31,21,http://tour-pedia.org/api/getReviewDetails?id=...
1,83456,fr,0,Je ne suis pas spécialiste des églises! Mais j...,2011-09-30,37,http://tour-pedia.org/api/getReviewDetails?id=...


In [11]:
df_base = pd.read_csv("paris_monuments_translated.csv")
print(df_base.shape)
df_base.head(2)

(88, 10)


,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw
0,Basilique Cathédrale de Saint-Denis,https://www.saint-denis-basilique.fr/,Discover the necropolis of the kings and queen...,17.0,Free for under 26s,From October to March\nMonday to Saturday: 10:...,coins | bank cards | cheques | Culture cheques...,"1, rue de la Légion d'Honneur, 93200 Saint-Denis",Cabin baggage ('cabines') allowed: maximum 40x...,Price: 17 €
1,Oppidum et musée archéologique d'Ensérune,https://www.enserune.fr/,Discover a major Gallo-Roman city located betw...,9.0,Free for those under 26 years old.,Please note: last access to the monument one h...,cash | bank cards | cheques | Culture cheques ...,"2901 route d’Ensérune, 34440 Nissan-lez-Ensérune",Drinking fountain | Toilets | Wheelchair | Pos...,Price: 9 €


# Merging the base dataset with the extended Tourpedia dataset
- Now we want to merge the base dataset with the extracted Tourpedia dataset.
- First we should identify the common rows between the two datasets based on the name, address.

In [12]:
import numpy as np
import unicodedata
from thefuzz import fuzz
import re
from urllib.parse import urlparse



FRENCH_STOP = {
    # very common French function words
    "de","du","des","la","le","les","d","l","au","aux","et","en","a",
    "un","une","pour","sur","sous","chez","dans","avec","sans","par",
    # address boilerplate
    "rue","avenue","av","boulevard","bd","route","rte","place","pl",
    "impasse","imp","square","sq","chemin","ch","allee","all",
    "saint","sainte","ste","st",
    # phone / contact noise
    "tel","telephone","t","contact"
}


LEGAL_WORDS = {
    "sarl","sas","sa","eurl","sasu","sci","scm","snc","selarl","selas",
    "association","assoc","societe","ste","ets","etablissements"
}

ADDR_ABBREV = {
    "av": "avenue",
    "bd": "boulevard",
    "bld": "boulevard",
    "rte": "route",
    "st": "saint",
    "ste": "sainte",
    "pl": "place",
    "imp": "impasse",
    "sq": "square",
}

def strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFKD", s)
    return "".join(c for c in s if not unicodedata.combining(c))

def normalize_text(s: str) -> str:
    if not s:
        return ""
    s = s.lower().strip()
    s = strip_accents(s)
    s = re.sub(r"[^a-z0-9]+", " ", s)   # punctuation -> space
    s = re.sub(r"\s+", " ", s).strip()
    s = " ".join([t for t in s.split() if t not in FRENCH_STOP])
    return s

def normalize_name(name: str) -> str:
    s = normalize_text(str(name) if name and name != "nan" else "")
    toks = [t for t in s.split() if t not in LEGAL_WORDS]
    return " ".join(toks)

def normalize_address(addr: str) -> str:
    s = normalize_text(str(addr) if addr and addr != "nan" else "")
    toks = [ADDR_ABBREV.get(t, t) for t in s.split()]
    toks = [t for t in toks if not t.isdigit() or len(t) == 5]
    return " ".join(toks)

def normalize_url(u: str) -> str:
    if not u:
        return ""
    u = u.strip()
    if not re.match(r"^[a-z]+://", u, re.I):
        u = "http://" + u
    p = urlparse(u)
    host = (p.netloc or "").lower()
    host = host[4:] if host.startswith("www.") else host
    path = re.sub(r"/+$", "", (p.path or "").lower())
    return host + path  # drop query/fragment

def make_fingerprint(name: str, address: str, url: str) -> str:
    return f"{normalize_name(name)} {normalize_address(address)} {normalize_url(url)}".strip()

def url_domain(u_norm: str) -> str:
    return u_norm.split("/")[0] if u_norm else ""


def token_set(s: str):
    return set(s.split()) if s else set()

def jaccard(a: str, b: str) -> float:
    A, B = token_set(a), token_set(b)
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

def score_pair(r1, r2, method: str = 'max'):
    if method not in {'max', 'avg'}:
        raise ValueError(f"Unknown method: {method}")
    n1, a1, u1 = r1["name_n"], r1["address_n"], r1["url_n"]
    n2, a2, u2 = r2["name_n"], r2["address_n"], r2["url_n"]

    # Gate 1: if both URLs exist and domains differ -> non-match and if domains same -> match
    if u1 and u2 and url_domain(u1) != url_domain(u2):
        return 0.0
    if u1 and u2 and url_domain(u1) == url_domain(u2):
        return 100.0

    # Gate 2: require minimal name token overlap to avoid generic matches
    if jaccard(n1, n2) < 0.15:   # tune 0.15–0.30 depending on your data
        return 0.0

    name_score = fuzz.WRatio(n1, n2) if (n1 and n2) else 0
    addr_score = fuzz.WRatio(a1, a2) if (a1 and a2) else 0

    scores = []
    if n1 and n2: scores.append(name_score)
    if a1 and a2: scores.append(addr_score)

    if not scores:
        return 0.0

    return max(scores) if method == 'max' else sum(scores) / len(scores)

In [13]:
import math

def distance_m(lat1, lon1, lat2, lon2):
    """Great-circle distance (Haversine) in meters between two lat/lon points."""
    R = 6371000.0  # Earth radius in meters

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

SIMILARITY_THRESHOLD = 70.0

df = df_tp_base_related_with_details.copy().rename(columns={'website': 'url'})

df["name_n"] = df["name"].fillna("").map(normalize_name)
df["address_n"] = df["address"].fillna("").map(normalize_address)
df["url_n"] = df["url"].fillna("").map(normalize_url)

similar_index_pairs = []
seen_rows = []

for idx, row in df.iterrows():
    is_duplicate = False
    max_dup_index = None
    max_dup_score = -1
    for idx_seen, seen_row in seen_rows:
        if distance_m(row['lat'], row['lng'], seen_row['lat'], seen_row['lng']) > 500:
            continue
        if (score := score_pair(row, seen_row)) > SIMILARITY_THRESHOLD:
            is_duplicate = True
            if score > max_dup_score:
                max_dup_score = score
                max_dup_index = idx_seen
    
    seen_rows.append((idx, row))
    if is_duplicate:
        similar_index_pairs.append((max_dup_index, idx))
        print("score:", max_dup_score)
        print(f"df1[{max_dup_index}]:", df.loc[max_dup_index, "name"], "|", df.loc[max_dup_index, "address"], "|", df.loc[max_dup_index, "url"])
        print(f"df2[{idx}]:", df.loc[idx, "name"], "|", df.loc[idx, "address"], "|", df.loc[idx, "url"])

df = df.drop(columns=["name_n", "address_n", "url_n"])

score: 100
df1[11]: Palais Royal | Place du Palais Royal | http://palais-royal.monuments-nationaux.fr
df2[69]: Place du Palais Royal | Place du Palais Royal | nan
score: 75
df1[47]: Église Sainte-Marie des Batignolles | Place du Docteur Félix Lobligeois | nan
df2[85]: Au coeur des Batignolles | 30 rue des Batignolles | nan
score: 100
df1[0]: Église de la Madeleine | Place de la Madeleine | http://www.eglise-lamadeleine.com
df2[88]: Place de la Madeleine | Place de la Madeleine | nan
score: 90
df1[84]: Paroisse Saint François de Molitor | 44 Rue Molitor | nan
df2[90]: Porte Molitor | Place de la Porte Molitor | nan
score: 86
df1[43]: Église Saint-Pierre de Montmartre | 2 rue du Mont-Cenis | nan
df2[96]: Marché Saint-Pierre | 2 Rue Charles Nodier | nan
score: 100.0
df1[11]: Palais Royal | Place du Palais Royal | http://palais-royal.monuments-nationaux.fr
df2[112]: Jardin du Palais Royal | 2 Place Colette | http://palais-royal.monuments-nationaux.fr
score: 100
df1[115]: Tombe de Jim Morri

## Similarity Checking with AI evaluation:
- I have evaluated 15 potential identical pairs in each prompt and used both GPT 5.2 Extended Thinking and Gemini Pro 3.
- I would accept the results only when both models agree on the same categorization.
- If there was a disagreement, I would manually check the pair and categorize it myself and also use GPT 5.2 Extended Thinking to re-evaluate due to it's access to google maps data.
prompt:
```markdown
I have provided a Pandas Dataframe with {30} rows from jupyter notebook cell output at the end of this prompt.
Each 2 consecutive rows are showing potential identical pairs.
I want you to evaluate this hypothesis for each of them and then categorize them by the following format.
# Confidently Identical:
{short explanation for each two pair in this category}
## output in the format of pair indexes:
[
    ({index1_1}, {index2_1}),
    ({index1_2}, {index2_2}),
    ...
]
# Confidently Different:
{short explanation for each two pair in this category}
## output in the format of pair indexes:
[
    ({index1_1}, {index2_1}),
    ({index1_2}, {index2_2}),
    ...
]
# Not Sure:
{short explanation for each two pair in this category}
## output in the format of pair indexes:
[
    ({index1_1}, {index2_1}),
    ({index1_2}, {index2_2}),
    ...
]


Dataframe:
{Dataframe output}
```

In [14]:
df_tp_base_related_with_details.iloc[
    np.array([[pair[0], pair[1]] for pair in similar_index_pairs]).flatten()
].loc[:, ["name", "address", "website", "lat", "lng", "subCategory", "Foursquare.url"]][0:30]

,name,address,website,lat,lng,subCategory,Foursquare.url
11,Palais Royal,Place du Palais Royal,http://palais-royal.monuments-nationaux.fr,48.863571,2.337019,Historic Site,https://foursquare.com/v/palais-royal/4adcda09...
69,Place du Palais Royal,Place du Palais Royal,NaN,48.862548,2.336751,Plaza,https://foursquare.com/v/place-du-palais-royal...
47,Église Sainte-Marie des Batignolles,Place du Docteur Félix Lobligeois,NaN,48.886797,2.317879,Church,https://foursquare.com/v/%C3%A9glise-saintemar...
85,Au coeur des Batignolles,30 rue des Batignolles,NaN,48.884916,2.320374,Park,https://foursquare.com/v/au-coeur-des-batignol...
0,Église de la Madeleine,Place de la Madeleine,http://www.eglise-lamadeleine.com,48.869754,2.324274,Church,https://foursquare.com/v/%C3%A9glise-de-la-mad...
88,Place de la Madeleine,Place de la Madeleine,NaN,48.869337,2.323952,Plaza,https://foursquare.com/v/place-de-la-madeleine...
84,Paroisse Saint François de Molitor,44 Rue Molitor,NaN,48.845331,2.259995,Church,https://foursquare.com/v/paroisse-saint-fran%C...
90,Porte Molitor,Place de la Porte Molitor,NaN,48.845839,2.256038,Plaza,https://foursquare.com/v/porte-molitor/4b70632...
43,Église Saint-Pierre de Montmartre,2 rue du Mont-Cenis,NaN,48.886707,2.342000,Church,https://foursquare.com/v/%C3%A9glise-saintpier...
96,Marché Saint-Pierre,2 Rue Charles Nodier,NaN,48.884373,2.344906,Art Gallery,https://foursquare.com/v/march%C3%A9-saintpier...


In [15]:
# Extracted Using AI evaluation:
# Should drop one of each pair (the priority is to keep the one that has reviews)
tp_identical_pairs = [
    (26, 272),
    (23, 347),
    (350, 351),
    (208, 412),
    (194, 429),
    (315, 436),
    (206, 267),
    (145, 461),
    (472, 473)
]

df_tp_base_related_with_details.iloc[
    np.array([[pair[0], pair[1]] for pair in tp_identical_pairs]).flatten()
]

,address,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews,Foursquare.url,Foursquare.usersCount,Foursquare.checkinsCount,Foursquare.tipCount,Foursquare.likes,website,numReviews
26,Place du Cardinal Amette,83529,48.850994,2.296472,Paris,Église Saint-Léon,4adcda0bf964a520883421e3,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/%C3%A9glise-saintl%C3...,25,33,0,0,NaN,0
272,Place du cardinal amette,87597,48.850519,2.296175,Paris,Paroisse Saint Léon,4d31afba8250f04d17f5ebe0,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/paroisse-saint-l%C3%A...,23,78,0,0,NaN,0
23,88 Rue de l'Assomption,83526,48.855703,2.266788,Paris,Notre-Dame de l'Assomption,4adcda0af964a5206d3421e3,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/notredame-de-lassompt...,18,22,0,0,NaN,0
347,90 Rue de l'Assomption,88648,48.855629,2.266488,Paris,Église Notre-Dame-de-l'Assomption de Passy,4e42fb3d483b13c708dcfa42,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/%C3%A9glise-notredame...,14,26,0,0,NaN,0
350,NaN,88702,48.840821,2.460270,Paris,Centre de Loisir Vincennes Nogent,4e4f6eadd164978659ca33fd,0,Park,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/centre-de-loisir-vinc...,2,12,0,0,NaN,0
351,NaN,88724,48.840819,2.459435,Paris,Centre de loisir Vincennes Bethoven,4e551a15ae60e3d6b7f5cd38,0,Park,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/centre-de-loisir-vinc...,2,4,0,0,NaN,0
208,Place Félix Eboué,86699,48.839611,2.395792,Paris,Place Félix Eboué,4c40ed7bd691c9b6a9ff8b0a,4,Plaza,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/place-f%C3%A9lix-ebou...,639,3920,7,12,http://paris.fr,7
412,Place Félix Eboué,90068,48.839576,2.396645,Paris,Place Félix Eboué,50059080e4b03f55b3f7eb77,0,Plaza,http://tour-pedia.org/api/getPlaceDetails?id=9...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/place-f%C3%A9lix-ebou...,46,171,0,1,NaN,0
194,6 Place de Bitche,86525,48.889201,2.379621,Paris,Église Saint-Jacques-Saint-Christophe de la Vi...,4c1e17628b3aa59325819b5f,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/%C3%A9glise-saintjacq...,16,22,0,0,NaN,0
429,NaN,90428,48.888727,2.379310,Paris,St Jacques,50714e4ae4b00d7436827916,0,Church,http://tour-pedia.org/api/getPlaceDetails?id=9...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/st-jacques/50714e4ae4...,8,12,0,0,NaN,0


In [16]:
# based on these columns quality ["name", "address", "website", "lat", "lng", "subCategory", "Foursquare.url"] we decided to keep the following indexes:
# but first we drop based on the reviews availability
tp_identical_pairs_priority_keep_inds = [
    26,
    347,
    350,
    208,
    194,
    315,
    206,
    145,
    472
]
tp_identical_pairs_drop_inds = []

for idx, pair in enumerate(tp_identical_pairs):
    priority_keep = tp_identical_pairs_priority_keep_inds[idx]
    other = pair[1] if pair[0] == priority_keep else pair[0]
    if df_tp_base_related_with_details.at[priority_keep, 'numReviews'] == 0 and df_tp_base_related_with_details.at[other, 'numReviews'] > 0:
        tp_identical_pairs_drop_inds.append(priority_keep)
    else:
        tp_identical_pairs_drop_inds.append(other)

print("Priority keeping indexes:", tp_identical_pairs_priority_keep_inds)
print("Dropping indexes:        ", tp_identical_pairs_drop_inds)        

Priority keeping indexes: [26, 347, 350, 208, 194, 315, 206, 145, 472]
Dropping indexes:         [272, 23, 351, 412, 429, 436, 267, 461, 472]


In [17]:
# Dorp Duplicates
df_tp_reviews_duplicate_free = df_tp_base_related_reviews.copy()[
    ~df_tp_base_related_reviews['monument_id'].isin(
        df_tp_base_related_with_details.loc[tp_identical_pairs_drop_inds, 'id'].tolist()   
    )
].reset_index(drop=True)

df_tp_duplicate_free = df_tp_base_related_with_details.copy().drop(
    index=tp_identical_pairs_drop_inds
).reset_index(drop=True)

In [18]:
print(df_tp_reviews_duplicate_free.shape)
df_tp_reviews_duplicate_free.head(2)

(2613, 7)


,monument_id,language,polarity,text,time,wordsCount,details_url
0,83456,fr,5,Une des trois plus laides églises de Paris ! M...,2011-08-31,21,http://tour-pedia.org/api/getReviewDetails?id=...
1,83456,fr,0,Je ne suis pas spécialiste des églises! Mais j...,2011-09-30,37,http://tour-pedia.org/api/getReviewDetails?id=...


In [19]:
print(df_tp_duplicate_free.shape)
df_tp_duplicate_free.head(2)

(477, 18)


,address,id,lat,lng,location,name,originalId,polarity,subCategory,details,reviews,Foursquare.url,Foursquare.usersCount,Foursquare.checkinsCount,Foursquare.tipCount,Foursquare.likes,website,numReviews
0,Place de la Madeleine,83456,48.869754,2.324274,Paris,Église de la Madeleine,4adcda09f964a520013421e3,4,Church,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/%C3%A9glise-de-la-mad...,1956,2839,9,28,http://www.eglise-lamadeleine.com,9
1,20 Avenue Rachel,83461,48.887549,2.330432,Paris,Cimetière de Montmartre,4adcda09f964a5200a3421e3,6,Cemetery,http://tour-pedia.org/api/getPlaceDetails?id=8...,http://tour-pedia.org/api/getReviewsByPlaceId?...,https://foursquare.com/v/cimeti%C3%A8re-de-mon...,636,1164,6,8,NaN,6


In [20]:
SIMILARITY_THRESHOLD = 70.0

df1 = df_base.copy()
df2 = df_tp_duplicate_free.rename(columns={'website': 'url'}).copy()

for df in (df1, df2):
    df["name_n"] = df["name"].fillna("").map(normalize_name)
    df["address_n"] = df["address"].fillna("").map(normalize_address)
    df["url_n"] = df["url"].fillna("").map(normalize_url)

similar_index_pairs = []

for idx2, r2 in df2.iterrows():
    max_similarity = -1
    sim_idx1 = None
    for idx1, r1 in df1.iterrows():
        similarity = score_pair(r2, r1, method='avg')
        if similarity > SIMILARITY_THRESHOLD:
            similar_index_pairs.append((idx1, idx2))
            print("similarity:", similarity)
            print(f"df1[{idx1}]:", df1.loc[idx1, "name"], "|", df1.loc[idx1, "address"], "|", df1.loc[idx1, "url"])
            print(f"df2[{idx2}]:", df2.loc[idx2, "name"], "|", df2.loc[idx2, "address"], "|", df2.loc[idx2, "url"])
        if similarity > max_similarity:
            max_similarity = similarity
            sim_idx1 = idx1
    df2.at[idx2, 'related_index'] = sim_idx1
    df2.at[idx2, 'similarity'] = max_similarity

df1 = df1.drop(columns=["name_n", "address_n", "url_n"])
df2 = df2.drop(columns=["name_n", "address_n", "url_n"])

similarity: 86.0
df1[36]: Maison et jardins de Georges Clemenceau | 76 rue Georges Clemenceau, 85520 Saint-Vincent-sur-Jard, Tel: 02 51 33 40 32, maison.clemenceau@monuments-nationaux.fr | https://www.maison-de-clemenceau.fr/
df2[103]: La Maison D'emile Zola | nan | nan
similarity: 73.0
df1[40]: Palais Jacques Cœur | 10 bis rue Jacques Cœur, 18000 Bourges | https://www.palais-jacques-coeur.fr/
df2[126]: Place Jacques Froment | Place Jacques Froment | nan
similarity: 75.0
df1[36]: Maison et jardins de Georges Clemenceau | 76 rue Georges Clemenceau, 85520 Saint-Vincent-sur-Jard, Tel: 02 51 33 40 32, maison.clemenceau@monuments-nationaux.fr | https://www.maison-de-clemenceau.fr/
df2[142]: Place Saint-Georges | Place Saint-Georges | nan
similarity: 86.0
df1[36]: Maison et jardins de Georges Clemenceau | 76 rue Georges Clemenceau, 85520 Saint-Vincent-sur-Jard, Tel: 02 51 33 40 32, maison.clemenceau@monuments-nationaux.fr | https://www.maison-de-clemenceau.fr/
df2[149]: La Maison Verte | nan

In [21]:
_df1_filtered = df1.loc[
    np.array([[pair[0], pair[1]] for pair in similar_index_pairs]).flatten().tolist()[::2]
].loc[:, 
    ["name", "address", "url"]
]

_df2_filtered = df2.loc[
    np.array([[pair[0], pair[1]] for pair in similar_index_pairs]).flatten().tolist()[1::2]
].rename(
    columns={"website": "url"}
).loc[:, 
    ["name", "address", "url", "location", "subCategory"]
]

_df1_filtered['sort_order'] = range(0, 2 * len(_df1_filtered), 2)
_df2_filtered['sort_order'] = range(1, 2 * len(_df2_filtered), 2)

pd.concat([
    _df1_filtered,
    _df2_filtered
]).sort_values('sort_order').drop(columns=['sort_order'])[30:]


,name,address,url,location,subCategory
30,Château d'If,"Île d'If, 13007 Marseille",https://www.chateau-if.fr/,NaN,NaN
367,Rue du Chateau des Rentiers,NaN,NaN,Paris,Plaza
37,Château de Castelnau-Bretenoux,"1 place Jean Moulierat, 46130 Prudhomat",https://www.castelnau-bretenoux.fr/,NaN,NaN
367,Rue du Chateau des Rentiers,NaN,NaN,Paris,Plaza
43,Château de Villeneuve-Lembron,63340 Villeneuve-Lembron,https://www.chateau-villeneuve-lembron.fr/,NaN,NaN
367,Rue du Chateau des Rentiers,NaN,NaN,Paris,Plaza
74,Domaine national du château de Coucy,"Rue du château, 02380 Coucy le Château Auffrique",https://www.chateau-coucy.fr/,NaN,NaN
367,Rue du Chateau des Rentiers,NaN,NaN,Paris,Plaza
77,Château de Gramont,"5 place du château, 82 120 GRAMONT",https://www.chateau-gramont.fr/,NaN,NaN
367,Rue du Chateau des Rentiers,NaN,NaN,Paris,Plaza


In [22]:
# Extracted Using AI evaluation:
# Should be considered while merging datasets:

identical_pairs = [
    (13, 150),
]

# Before merging
- We have decide to keep all the columns form the base dataset and the following columns from the Tourpedia dataset:
    - name                           (exists in base dataset)
    - address                        (exists in base dataset)
    - website ->  url (exists in base dataset)
    - city
    - subCategory -> category
    - lat
    - lng
    - id -> Tourpedia_id
- The data related to the Foursquare will be kept in anouther dataframe called df_tp_reviews_duplicate_free with the following columns:
    - id -> Tourpedia_id
    - originalId -> original_id
    - Foursquare.url -> Foursquare_url
    - Foursquare.usersCount	-> Foursquare_users_count
    - Foursquare.checkinsCount -> Foursquare_checkins_count
    - Foursquare.tipCount -> Foursquare_tip_count
    - Foursquare.likes -> Foursquare_likes
- For the reviews dataset we will keep the columns like below:
    - monument_id -> Tourpedia_id
    - language
    - polarity
    - text
    - time
    - wordsCount -> word_count
    - details_url -> tokenized_text_url

In [23]:
df_tp_fs = df_tp_duplicate_free.rename(columns={
    "id": "Tourpedia_id",
    "originalId": "original_id",
    "Foursquare.url": "Foursquare_url",
    "Foursquare.usersCount": "Foursquare_users_count",
    "Foursquare.checkinsCount": "Foursquare_checkins_count",
    "Foursquare.tipCount": "Foursquare_tip_count",
    "Foursquare.likes": "Foursquare_likes"
}).loc[:, [
    "Tourpedia_id",
    "original_id",
    "Foursquare_url",
    "Foursquare_users_count",
    "Foursquare_checkins_count",
    "Foursquare_tip_count",
    "Foursquare_likes"
]].copy()
df_tp_fs.head(2)

,Tourpedia_id,original_id,Foursquare_url,Foursquare_users_count,Foursquare_checkins_count,Foursquare_tip_count,Foursquare_likes
0,83456,4adcda09f964a520013421e3,https://foursquare.com/v/%C3%A9glise-de-la-mad...,1956,2839,9,28
1,83461,4adcda09f964a5200a3421e3,https://foursquare.com/v/cimeti%C3%A8re-de-mon...,636,1164,6,8


In [24]:
df_tp_fs_reviews = df_tp_reviews_duplicate_free.rename(columns={
    "monument_id": "Tourpedia_id",
    "wordsCount": "words_count",
    "details_url": "tokenized_text_url"
}).copy()
df_tp_fs_reviews.head(2)

,Tourpedia_id,language,polarity,text,time,words_count,tokenized_text_url
0,83456,fr,5,Une des trois plus laides églises de Paris ! M...,2011-08-31,21,http://tour-pedia.org/api/getReviewDetails?id=...
1,83456,fr,0,Je ne suis pas spécialiste des églises! Mais j...,2011-09-30,37,http://tour-pedia.org/api/getReviewDetails?id=...


In [25]:
df_base_updated = df_base.copy()
df_base_updated[["city", "category", "lat", "lng", "Tourpedia_id"]] = pd.NA
df_base_updated = df_base_updated.astype({
    "lat": "Float64",
    "lng": "Float64",
    "Tourpedia_id": "Int64"
})

df_base_updated.loc[
    np.array(identical_pairs).flatten().tolist()[0::2],
    ["city", "category", "lat", "lng", "Tourpedia_id"]
] = df_tp_duplicate_free.loc[
    np.array(identical_pairs).flatten().tolist()[1::2],
    ["location", "subCategory", "lat", "lng", "id"]
].rename(columns={
    "location": "city",
    "subCategory": "category",
    "id": "Tourpedia_id"
}).values

df_base_updated = pd.concat([
    df_base_updated,
    df_tp_duplicate_free.loc[:,[
        "name",
        "address",
        "website",
        "location",
        "subCategory",
        "lat",
        "lng",
        "id"
    ]].rename(columns={
        "id": "Tourpedia_id",
        "website": "url",
        "subCategory": "category",
        "location": "city"
    }).drop(
        index=np.array(identical_pairs).flatten().tolist()[1::2]
    ).copy()    
]).reset_index(drop=True)
df_base_updated.head(2)


,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw,city,category,lat,lng,Tourpedia_id
0,Basilique Cathédrale de Saint-Denis,https://www.saint-denis-basilique.fr/,Discover the necropolis of the kings and queen...,17.0,Free for under 26s,From October to March\nMonday to Saturday: 10:...,coins | bank cards | cheques | Culture cheques...,"1, rue de la Légion d'Honneur, 93200 Saint-Denis",Cabin baggage ('cabines') allowed: maximum 40x...,Price: 17 €,<NA>,<NA>,<NA>,<NA>,<NA>
1,Oppidum et musée archéologique d'Ensérune,https://www.enserune.fr/,Discover a major Gallo-Roman city located betw...,9.0,Free for those under 26 years old.,Please note: last access to the monument one h...,cash | bank cards | cheques | Culture cheques ...,"2901 route d’Ensérune, 34440 Nissan-lez-Ensérune",Drinking fountain | Toilets | Wheelchair | Pos...,Price: 9 €,<NA>,<NA>,<NA>,<NA>,<NA>


# Saving the merged datasets

In [ ]:
# df_base_updated.to_csv("merged_datasets/france_monuments_merged.csv", index=False)
# df_tp_fs.to_csv("merged_datasets/Tourpedia_Foursquare_data.csv", index=False)
# df_tp_fs_reviews.to_csv("merged_datasets/Tourpedia_Foursquare_reviews.csv", index=False)